In [54]:
import pandas as pd
import numpy as np


# string processing
import re
from keras.utils.np_utils import to_categorical
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics


In [14]:
# import the true label

df_merge_quality = pd.read_csv('data/US_patent_abstract_5000_2015_with_title_1_5y.csv')

In [34]:
y_test_true = df_merge_quality['quality_rank'][4000:].values
print(y_test_true.shape)
y_test_true[:10]

(1000,)


array([0, 1, 0, 0, 0, 1, 1, 1, 1, 1])

In [32]:
# double check the label distribution
# Validation set

count = len(y_test_true)
positive = sum(y_test_true)
negative = sum(x == 0 for x in y_test_true)
    
print(f'Total Sample Count = {count} \nPositive Label Count = {positive} \nNegative Label Count = {negative} \nPositive Class Ratio = {positive/count}  \nNegative Class Ratio = {negative/count}')


Total Sample Count = 1000 
Positive Label Count = 405 
Negative Label Count = 595 
Positive Class Ratio = 0.405  
Negative Class Ratio = 0.595


In [138]:
test_read1 = pd.read_csv("Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv", 
                        names = ['Class 0', 'Class 1'], 
                        sep=" ")
test_read1

,Class 0,Class 1
0,0.847997,0.130727
1,0.332160,0.667886
2,0.236171,0.788574
3,0.887799,0.081256
4,0.828767,0.121260
...,...,...
995,0.540695,0.398545
996,0.932154,0.059405
997,0.195182,0.780047
998,0.534664,0.487115


In [139]:
test_read2 = pd.read_csv("Prediction_Output/BERT_fine_tune_5yr_abstract_title_dev_prob-Copy1.csv", 
                        names = ['Class 0', 'Class 1'], 
                        sep=" ")
test_read2

,Class 0,Class 1
0,0.180449,0.816414
1,0.724672,0.255530
2,0.409250,0.577868
3,0.927518,0.081868
4,0.979033,0.020911
...,...,...
995,0.838626,0.148926
996,0.833403,0.162426
997,0.180923,0.784803
998,0.193379,0.816204


In [140]:
test_read1 * 0.5 + test_read2 * 0.5

,Class 0,Class 1
0,0.514223,0.473570
1,0.528416,0.461708
2,0.322710,0.683221
3,0.907659,0.081562
4,0.903900,0.071086
...,...,...
995,0.689661,0.273735
996,0.882778,0.110915
997,0.188053,0.782425
998,0.364021,0.651659


In [141]:
(test_read1 + test_read2) * 0.5

,Class 0,Class 1
0,0.514223,0.473570
1,0.528416,0.461708
2,0.322710,0.683221
3,0.907659,0.081562
4,0.903900,0.071086
...,...,...
995,0.689661,0.273735
996,0.882778,0.110915
997,0.188053,0.782425
998,0.364021,0.651659


In [67]:
# https://cmdlinetips.com/2018/12/how-to-loop-through-pandas-rows-or-how-to-iterate-over-pandas-rows/#:~:text=Pandas%20has%20iterrows()%20function,the%20content%20of%20the%20iterator.
for i in test_read.itertuples():
    print(i._1, i._2)

0.8479969501495361 0.1307267248630523
0.3321598172187805 0.6678864359855652
0.23617091774940485 0.7885740995407104
0.8877992033958436 0.08125635981559755
0.828767418861389 0.12126028537750243
0.6365858316421509 0.30468297004699707
0.7011922597885132 0.24463146924972534
0.8038750290870667 0.1310310065746307
0.8003423810005189 0.19075867533683774
0.963689565658569 0.028326630592346195
0.8826712965965272 0.09366616606712339
0.738400399684906 0.2574821710586548
0.799733817577362 0.19859188795089722
0.9297205209732058 0.051439821720123284
0.788860023021698 0.2169454097747803
0.8776333332061766 0.13374078273773193
0.8845688104629517 0.08181545138359068
0.8796502351760864 0.11024996638298033
0.9105886220932006 0.1002819538116455
0.7579448223114011 0.25525808334350586
0.3531104922294617 0.6041271090507507
0.8072425723075866 0.19061025977134705
0.5108556747436525 0.4793073534965516
0.41507914662361145 0.6663478016853333
0.8322030305862426 0.1711343824863434
0.6264361143112183 0.3068888187408447

In [66]:
# https://cmdlinetips.com/2018/12/how-to-loop-through-pandas-rows-or-how-to-iterate-over-pandas-rows/#:~:text=Pandas%20has%20iterrows()%20function,the%20content%20of%20the%20iterator.
for i in test_read.itertuples():
    print(np.argmax([i._1, i._2]))

0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0


In [79]:
temp_pred0 = []
for i in test_read.itertuples():
    temp_pred0.append(np.argmax([i._1, i._2]))
temp_pred0    
    

[0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [80]:
accuracy = metrics.accuracy_score(y_test_true, temp_pred0)
accuracy

0.603

In [48]:
temp_pred = []
with open("Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv") as f1:
    for line in f1.readlines():
        print(line.split())

['8.479969501495361328e-01', '1.307267248630523682e-01']
['3.321598172187805176e-01', '6.678864359855651855e-01']
['2.361709177494049072e-01', '7.885740995407104492e-01']
['8.877992033958435059e-01', '8.125635981559753418e-02']
['8.287674188613891602e-01', '1.212602853775024414e-01']
['6.365858316421508789e-01', '3.046829700469970703e-01']
['7.011922597885131836e-01', '2.446314692497253418e-01']
['8.038750290870666504e-01', '1.310310065746307373e-01']
['8.003423810005187988e-01', '1.907586753368377686e-01']
['9.636895656585693359e-01', '2.832663059234619141e-02']
['8.826712965965270996e-01', '9.366616606712341309e-02']
['7.384003996849060059e-01', '2.574821710586547852e-01']
['7.997338175773620605e-01', '1.985918879508972168e-01']
['9.297205209732055664e-01', '5.143982172012329102e-02']
['7.888600230216979980e-01', '2.169454097747802734e-01']
['8.776333332061767578e-01', '1.337407827377319336e-01']
['8.845688104629516602e-01', '8.181545138359069824e-02']
['8.796502351760864258e-01', '1

In [73]:
temp_pred = []
with open("Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv") as f1:
    for line in f1.readlines():
        print([float(x) for x in line.split()])

[0.8479969501495361, 0.13072672486305237]
[0.3321598172187805, 0.6678864359855652]
[0.2361709177494049, 0.7885740995407104]
[0.8877992033958435, 0.08125635981559753]
[0.8287674188613892, 0.12126028537750244]
[0.6365858316421509, 0.30468297004699707]
[0.7011922597885132, 0.24463146924972534]
[0.8038750290870667, 0.13103100657463074]
[0.8003423810005188, 0.19075867533683777]
[0.9636895656585693, 0.02832663059234619]
[0.8826712965965271, 0.09366616606712341]
[0.738400399684906, 0.2574821710586548]
[0.7997338175773621, 0.19859188795089722]
[0.9297205209732056, 0.05143982172012329]
[0.788860023021698, 0.21694540977478027]
[0.8776333332061768, 0.13374078273773193]
[0.8845688104629517, 0.0818154513835907]
[0.8796502351760864, 0.11024996638298035]
[0.9105886220932007, 0.10028195381164551]
[0.7579448223114014, 0.25525808334350586]
[0.35311049222946167, 0.6041271090507507]
[0.8072425723075867, 0.19061025977134705]
[0.5108556747436523, 0.4793073534965515]
[0.41507914662361145, 0.6663478016853333]

In [77]:
temp_pred2 = []
with open("Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv") as f1:
    for line in f1.readlines():
        temp_pred2.append(np.argmax([float(x) for x in line.split()]))
temp_pred2

[0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [78]:
accuracy = metrics.accuracy_score(y_test_true, temp_pred2)
accuracy

0.603

In [81]:
metrics.precision_score(y_test_true, temp_pred2)

0.5307692307692308

In [82]:
metrics.recall_score(y_test_true, temp_pred2)

0.17037037037037037

In [83]:
metrics.f1_score(y_test_true, temp_pred2)

0.25794392523364484

In [85]:
metrics.roc_auc_score(y_test_true, temp_pred2)

0.5339246809835044

In [52]:
temp_pred = []
with open("Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv") as f1:
    for line in f1.readlines():
        temp_pred.append(np.argmax(line.split()))
len(temp_pred)

1000

In [57]:
temp_pred

[0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [56]:
accuracy = metrics.accuracy_score(y_test_true, temp_pred)
accuracy

0.596

In [58]:
metrics.precision_score(y_test_true, temp_pred)

0.5029239766081871

In [37]:
for row in test_read:
    print(row)

Class 0
Class 1


In [3]:
# https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory

import os

directory = "Prediction_Output"

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        print(os.path.join(directory, filename))
        continue
    else:
        continue

Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv
Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv
Prediction_Output/HATT.csv
Prediction_Output/Distil-BERT_5yr_claims_dev_prob-Copy1.csv
Prediction_Output/BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv
Prediction_Output/CNN_5yr_claims_dev_prob-Copy1.csv
Prediction_Output/BERT_fine_tune_5yr_abstract_title_dev_prob-Copy1.csv
Prediction_Output/BERT_CNN_5yr_claims_dev_prob-Copy1.csv
Prediction_Output/LSTM.csv
Prediction_Output/TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv
Prediction_Output/CNN_5yr_abstract_title_dev_prob-Copy1.csv
Prediction_Output/HATT_50k_5y.csv
Prediction_Output/LSTM_5y_50k.csv


In [4]:
# create a list holding file names, so it's easier to iterate / combinations later

In [163]:
file_name = []

for filename in os.listdir(directory):
    if filename.endswith(".csv") and not('50k' in filename): # not include 50k version for now
        file_name.append(os.path.join(directory, filename))
       

In [164]:
file_name

['Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
 'Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv',
 'Prediction_Output/HATT.csv',
 'Prediction_Output/Distil-BERT_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/CNN_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/BERT_fine_tune_5yr_abstract_title_dev_prob-Copy1.csv',
 'Prediction_Output/BERT_CNN_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/LSTM.csv',
 'Prediction_Output/TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv',
 'Prediction_Output/BERT_fine_tune_5yr_claims_dev_prob_last_n_words-Copy1.csv',
 'Prediction_Output/CNN_5yr_abstract_title_dev_prob-Copy1.csv']

In [156]:
13*12 / 2

78.0

### For summary purpose, show the metrics for all files

In [91]:
df = pd.DataFrame(columns=['A', 'B'])

values_to_add = {'A': 1, 'B': 2}
# row_to_add = pd.Series(values_to_add, name='x') # => I can skip this step if below you add ignore_index=True

df = df.append(values_to_add, ignore_index = True)
df

,A,B
0,1,2


In [157]:
summary_df = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision', 'Recall','F1','AUC'])

for file in file_name:
    temp_pred_result = []
    with open(file) as f1:
        for line in f1.readlines():
            temp_pred_result.append(np.argmax([float(x) for x in line.split()]))
    #print(file)
    
    accuracy = metrics.accuracy_score(y_test_true, temp_pred_result)
    precision = metrics.precision_score(y_test_true, temp_pred_result)
    recall = metrics.recall_score(y_test_true, temp_pred_result)
    f1 = metrics.f1_score(y_test_true, temp_pred_result)
    auc = metrics.roc_auc_score(y_test_true, temp_pred_result)
    
    if '/' in file: # to show only the filename not the path
        file_name = file.split('/')[-1]
    else:
        file_name = file
    #file_name = if ('/' in file): file[file.find('/'):] else: file 
    
    row_to_add = {'Model': file_name, 'Accuracy': accuracy, 'Precision':precision, 'Recall': recall,'F1': f1,'AUC': auc}
    
    summary_df = summary_df.append(row_to_add, ignore_index = True)
    

In [158]:
summary_df

,Model,Accuracy,Precision,Recall,F1,AUC
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,0.600,0.531646,0.103704,0.173554,0.520759
1,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.603,0.530769,0.170370,0.257944,0.533925
2,HATT.csv,0.699,0.692593,0.461728,0.554074,0.661116
3,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.627,0.602564,0.232099,0.335116,0.563949
4,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.576744,0.306173,0.400000,0.576616
5,CNN_5yr_claims_dev_prob-Copy1.csv,0.625,0.548701,0.417284,0.474053,0.591835
6,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.616,0.538462,0.362963,0.433628,0.575599
7,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.604,0.513846,0.412346,0.457534,0.573400
8,LSTM.csv,0.681,0.622857,0.538272,0.577483,0.658211
9,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.618,0.632184,0.135802,0.223577,0.541010


In [ ]:
summary_df.sort_values(by='Accuracy')

In [168]:
12*11/2

66.0

In [78]:
accuracy = metrics.accuracy_score(y_test_true, temp_pred2)
accuracy

0.603

In [81]:
metrics.precision_score(y_test_true, temp_pred2)

0.5307692307692308

In [82]:
metrics.recall_score(y_test_true, temp_pred2)

0.17037037037037037

In [83]:
metrics.f1_score(y_test_true, temp_pred2)

0.25794392523364484

In [85]:
metrics.roc_auc_score(y_test_true, temp_pred2)

0.5339246809835044

### Ensemble Strategy: 
The strategy of ensemble is to average the predicted probability for both Class 0 and Class 1 from  two different models, then take argmax to determine the class by the larger average probability.

In [165]:
# To generate all combinatition of pairs of models
# https://www.geeksforgeeks.org/itertools-combinations-module-python-print-possible-combinations/
from itertools import combinations

pair_iterator = list(combinations(file_name, 2))

In [166]:
len(pair_iterator)

66

In [159]:
pair_iterator

[('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/HATT.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/Distil-BERT_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/CNN_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_fine_tune_5yr_abstract_title_dev_prob-Copy1.csv'),
 ('Prediction_Output/TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-Copy1.csv',
  'Prediction_Output/BERT_CNN_5yr_claims_dev_prob-Copy1.csv'),
 ('Prediction_Out

In [149]:
df_combine = test_read1 * 0.5 + test_read2 * 0.5

In [151]:
temp_pred0 = []
for i in df_combine.itertuples():
    print(i)
    #temp_pred0.append(np.argmax([i._1, i._2]))
#temp_pred0    

Pandas(Index=0, _1=0.514223113656044, _2=0.4735701233148575)
Pandas(Index=1, _1=0.5284160077571869, _2=0.4617079794406891)
Pandas(Index=2, _1=0.3227103799581527, _2=0.6832211315631866)
Pandas(Index=3, _1=0.907658785581589, _2=0.08156231045722961)
Pandas(Index=4, _1=0.9039002060890198, _2=0.07108557224273682)
Pandas(Index=5, _1=0.7475932240486145, _2=0.22592869400978088)
Pandas(Index=6, _1=0.7373192012310028, _2=0.23788535594940186)
Pandas(Index=7, _1=0.8143908083438873, _2=0.14788471162319183)
Pandas(Index=8, _1=0.6771859526634216, _2=0.3254529535770416)
Pandas(Index=9, _1=0.9481260180473325, _2=0.04455211758613586)
Pandas(Index=10, _1=0.9173003733158112, _2=0.07029126584529875)
Pandas(Index=11, _1=0.5056843757629395, _2=0.4857106506824494)
Pandas(Index=12, _1=0.8708793222904205, _2=0.12936103343963623)
Pandas(Index=13, _1=0.9322277307510377, _2=0.05282709002494812)
Pandas(Index=14, _1=0.5951706618070602, _2=0.40556401014328)
Pandas(Index=15, _1=0.6921009123325348, _2=0.326482474803924

In [160]:
def clean_filename(filepath):
    '''
    filepah: to pass in string
    '''
    
    if '/' in filepath: # to show only the filename not the path
        file_name = filepath.split('/')[-1]
    else:
        file_name = filepath
    
    return file_name

In [172]:
pd.set_option('display.max_rows', 100)

In [173]:
# for this purpose, since I need to average two files together, I feel it's easier to use pandas dataframe and use vectorized operation

df_ensemble_compare = pd.DataFrame(columns = ['Model 1', 'Model 2', 'Accuracy', 'Precision', 'Recall','F1','AUC'])

for files in pair_iterator:
    temp_pred_result = []
    
    
    # temporary hold of data from two files
    df_file_1 = pd.read_csv(files[0], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")

    df_file_2 = pd.read_csv(files[1], 
                            names = ['Class 0', 'Class 1'], 
                            sep=" ")
    
    # average two files
    df_combine = df_file_1 * 0.5 + df_file_2 * 0.5

    # compute the new predicted output
    for i in df_combine.itertuples():
        temp_pred_result.append(np.argmax([i._1, i._2]))
    temp_pred_result
    
    accuracy = metrics.accuracy_score(y_test_true, temp_pred_result)
    precision = metrics.precision_score(y_test_true, temp_pred_result)
    recall = metrics.recall_score(y_test_true, temp_pred_result)
    f1 = metrics.f1_score(y_test_true, temp_pred_result)
    auc = metrics.roc_auc_score(y_test_true, temp_pred_result)
    
    
    row_to_add = {'Model 1': clean_filename(files[0]), 'Model 2': clean_filename(files[1]), 'Accuracy': accuracy, 'Precision':precision, 'Recall': recall,'F1': f1,'AUC': auc}
    
    df_ensemble_compare = df_ensemble_compare.append(row_to_add, ignore_index = True)
    
df_ensemble_compare

,Model 1,Model 2,Accuracy,Precision,Recall,F1,AUC
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.608,0.571429,0.128395,0.209677,0.531424
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,0.698,0.784530,0.350617,0.484642,0.642536
2,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.618,0.666667,0.113580,0.194093,0.537462
3,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.627907,0.200000,0.303371,0.559664
4,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,CNN_5yr_claims_dev_prob-Copy1.csv,0.639,0.582707,0.382716,0.461997,0.598081
5,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.619,0.558824,0.281481,0.374384,0.565110
6,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.614,0.569343,0.192593,0.287823,0.546716
7,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,LSTM.csv,0.677,0.670833,0.397531,0.499225,0.632379
8,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.614,0.633803,0.111111,0.189076,0.533707
9,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.668,0.646586,0.397531,0.492355,0.624816


In [175]:
print("Rank by Accuracy")
df_ensemble_compare.sort_values(by = "Accuracy", ascending = False)

Rank by Accuracy


,Model 1,Model 2,Accuracy,Precision,Recall,F1,AUC
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,0.698,0.784530,0.350617,0.484642,0.642536
27,HATT.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.691,0.758065,0.348148,0.477157,0.636259
22,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.691,0.695122,0.422222,0.525346,0.648086
26,HATT.csv,LSTM.csv,0.690,0.644377,0.523457,0.577657,0.663409
25,HATT.csv,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.683,0.648649,0.474074,0.547789,0.649642
21,HATT.csv,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.682,0.693333,0.385185,0.495238,0.634609
11,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,0.681,0.712871,0.355556,0.474465,0.629038
60,LSTM.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.680,0.680851,0.395062,0.500000,0.634506
28,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.679,0.637255,0.481481,0.548523,0.647463
41,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,LSTM.csv,0.678,0.647687,0.449383,0.530612,0.641498


In [176]:
print("Rank by Precision")
df_ensemble_compare.sort_values(by = "Precision", ascending = False)

Rank by Precision


,Model 1,Model 2,Accuracy,Precision,Recall,F1,AUC
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,0.698,0.784530,0.350617,0.484642,0.642536
27,HATT.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.691,0.758065,0.348148,0.477157,0.636259
11,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,HATT.csv,0.681,0.712871,0.355556,0.474465,0.629038
22,HATT.csv,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.691,0.695122,0.422222,0.525346,0.648086
21,HATT.csv,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.682,0.693333,0.385185,0.495238,0.634609
60,LSTM.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.680,0.680851,0.395062,0.500000,0.634506
17,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,LSTM.csv,0.671,0.675926,0.360494,0.470209,0.621423
19,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.670,0.674419,0.358025,0.467742,0.620189
7,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,LSTM.csv,0.677,0.670833,0.397531,0.499225,0.632379
35,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.626,0.670330,0.150617,0.245968,0.550099


In [216]:
# Insert a new column
summary_df_copy = summary_df.copy()

summary_df_copy

,Model,Accuracy,Precision,Recall,F1,AUC
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,0.600,0.531646,0.103704,0.173554,0.520759
1,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.603,0.530769,0.170370,0.257944,0.533925
2,HATT.csv,0.699,0.692593,0.461728,0.554074,0.661116
3,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.627,0.602564,0.232099,0.335116,0.563949
4,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.576744,0.306173,0.400000,0.576616
5,CNN_5yr_claims_dev_prob-Copy1.csv,0.625,0.548701,0.417284,0.474053,0.591835
6,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.616,0.538462,0.362963,0.433628,0.575599
7,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.604,0.513846,0.412346,0.457534,0.573400
8,LSTM.csv,0.681,0.622857,0.538272,0.577483,0.658211
9,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.618,0.632184,0.135802,0.223577,0.541010


In [219]:
# This is simple and clean, but the new column always add in the end, 
summary_df_copy['Model 2'] = ""
summary_df_copy
# Note: I tried the .insert method, but it keeps not output anything. Not sure why...
#summary_df_copy2 = summary_df_copy.insert(1, "Model_2", [1 for i in range(len(summary_df_copy.index))], True)

,Model,Accuracy,Precision,Recall,F1,AUC,Model 2
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,0.600,0.531646,0.103704,0.173554,0.520759,
1,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.603,0.530769,0.170370,0.257944,0.533925,
2,HATT.csv,0.699,0.692593,0.461728,0.554074,0.661116,
3,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.627,0.602564,0.232099,0.335116,0.563949,
4,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.576744,0.306173,0.400000,0.576616,
5,CNN_5yr_claims_dev_prob-Copy1.csv,0.625,0.548701,0.417284,0.474053,0.591835,
6,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.616,0.538462,0.362963,0.433628,0.575599,
7,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.604,0.513846,0.412346,0.457534,0.573400,
8,LSTM.csv,0.681,0.622857,0.538272,0.577483,0.658211,
9,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.618,0.632184,0.135802,0.223577,0.541010,


In [213]:
summary_df_copy = summary_df_copy.rename(columns={"Model": "Model 1"})

In [214]:
summary_df_copy

,Model 1,Accuracy,Precision,Recall,F1,AUC,Model 2
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,0.600,0.531646,0.103704,0.173554,0.520759,
1,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.603,0.530769,0.170370,0.257944,0.533925,
2,HATT.csv,0.699,0.692593,0.461728,0.554074,0.661116,
3,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.627,0.602564,0.232099,0.335116,0.563949,
4,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.576744,0.306173,0.400000,0.576616,
5,CNN_5yr_claims_dev_prob-Copy1.csv,0.625,0.548701,0.417284,0.474053,0.591835,
6,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.616,0.538462,0.362963,0.433628,0.575599,
7,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.604,0.513846,0.412346,0.457534,0.573400,
8,LSTM.csv,0.681,0.622857,0.538272,0.577483,0.658211,
9,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.618,0.632184,0.135802,0.223577,0.541010,


In [215]:
# .concat will merge based on the column header (not by the position), but perhaps there is some argument to adjust that?
pd.concat([df_ensemble_compare, summary_df_copy])

,Model 1,Model 2,Accuracy,Precision,Recall,F1,AUC
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.608,0.571429,0.128395,0.209677,0.531424
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,0.698,0.784530,0.350617,0.484642,0.642536
2,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.618,0.666667,0.113580,0.194093,0.537462
3,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.627907,0.200000,0.303371,0.559664
4,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,CNN_5yr_claims_dev_prob-Copy1.csv,0.639,0.582707,0.382716,0.461997,0.598081
5,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.619,0.558824,0.281481,0.374384,0.565110
6,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.614,0.569343,0.192593,0.287823,0.546716
7,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,LSTM.csv,0.677,0.670833,0.397531,0.499225,0.632379
8,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.614,0.633803,0.111111,0.189076,0.533707
9,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.668,0.646586,0.397531,0.492355,0.624816


In [217]:
pd.concat([df_ensemble_compare, summary_df_copy])

,Model 1,Model 2,Accuracy,Precision,Recall,F1,AUC,Model
0,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_abstract_title_dev_prob-Copy1.csv,0.608,0.571429,0.128395,0.209677,0.531424,NaN
1,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,HATT.csv,0.698,0.784530,0.350617,0.484642,0.642536,NaN
2,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,Distil-BERT_5yr_claims_dev_prob-Copy1.csv,0.618,0.666667,0.113580,0.194093,0.537462,NaN
3,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_claims_dev_prob-Copy1.csv,0.628,0.627907,0.200000,0.303371,0.559664,NaN
4,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,CNN_5yr_claims_dev_prob-Copy1.csv,0.639,0.582707,0.382716,0.461997,0.598081,NaN
5,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_abstract_title_dev_prob-Cop...,0.619,0.558824,0.281481,0.374384,0.565110,NaN
6,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_CNN_5yr_claims_dev_prob-Copy1.csv,0.614,0.569343,0.192593,0.287823,0.546716,NaN
7,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,LSTM.csv,0.677,0.670833,0.397531,0.499225,0.632379,NaN
8,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,TFIDF_NaiveBayes_5yr_claims_dev_prob-Copy1.csv,0.614,0.633803,0.111111,0.189076,0.533707,NaN
9,TFIDF_NaiveBayes_5yr_abstract_title_dev_prob-C...,BERT_fine_tune_5yr_claims_dev_prob_last_n_word...,0.668,0.646586,0.397531,0.492355,0.624816,NaN


In [188]:
[None for i in range(len(summary_df_copy.index))]

[None, None, None, None, None, None, None, None, None, None, None, None]

In [191]:
import pandas as pd 
  
# Define a dictionary containing Students data 
data = {'Name': ['Jai', 'Princi', 'Gaurav', 'Anuj'], 
        'Height': [5.1, 6.2, 5.1, 5.2], 
        'Qualification': ['Msc', 'MA', 'Msc', 'Msc']} 
  
# Convert the dictionary into DataFrame 
df = pd.DataFrame(data) 
  
# Using DataFrame.insert() to add a column 
df.insert(2, "Age", [21, 23, 24, 21], True) 
  
# Observe the result 
df 

,Name,Height,Age,Qualification
0,Jai,5.1,21,Msc
1,Princi,6.2,23,MA
2,Gaurav,5.1,24,Msc
3,Anuj,5.2,21,Msc


In [185]:
summary_df_copy

In [ ]:
# Concatenate 2 dataframes
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

# 